## Create original Postal Code Dataframe

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# Get table with BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url).text
soup = BeautifulSoup(page, 'lxml')
table = soup.find("table",{"class":"wikitable sortable"})

CanadaData = []

# Create List
for tr in table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    # Remove new line characters
    row = [word.replace('\n','') for word in row]
    CanadaData.append(row)

del CanadaData[0]    

# Populate new dataframe with list
df = pd.DataFrame(data = CanadaData, columns=['PostalCode', 'Borough', 'Neighborhood'])

# Remove Unassigned Boroughs
df = df[df['Borough'] != 'Not assigned']

# Combine Neighborhoods for each Postal Code
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Match Unassigned Neighborhoods to Boroughs
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']

# Import Geospatial Data
LatLong = pd.read_csv('http://cocl.us/Geospatial_data')

# Join Postal Code and Latitude-Longitude Data
df = pd.merge(df,LatLong,left_on='PostalCode', right_on='Postal Code')

# Remove Duplicate Column
df.drop(['Postal Code'], axis=1, inplace=True)

## Import Libraries

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge


## Get the latitude and longitude values of Toronto

In [3]:
address = 'Toronto, CAN'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.6604619, -79.3861563.


## Create Toronto Neighborhood Map

In [4]:
# Create new dataframe for Boroughs that contain "Toronto"
Toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

# Create a map centered around Toronto
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add the Toronto Neighborhoods as blue circle markers
for lat, lng, label in zip(Toronto_df.Latitude, Toronto_df.Longitude, Toronto_df.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(Toronto_map)

# display map
Toronto_map

## Show number of distinct postal code neighborhoods by Toronto boroughs

In [5]:
Toronto_grouped = Toronto_df.groupby('Borough').count().reset_index()
Toronto_grouped.drop(['PostalCode', 'Latitude', 'Longitude'], axis=1, inplace=True)
Toronto_grouped.rename({'Neighborhood': '# of Neighborhoods'}, axis=1, inplace=True)

Toronto_grouped

,Borough,# of Neighborhoods
0,Central Toronto,9
1,Downtown Toronto,18
2,East Toronto,5
3,West Toronto,6
